In [1]:
#Import Libraries

import requests
import bs4
import mysql.connector
import pandas as pd
import json


In [47]:
url_source = "https://www.jumia.co.ke/phones-tablets/samsung/#catalog-listing"
url_data = requests.get(url_source)

html_content = bs4.BeautifulSoup(url_data.content)

all_items = html_content("article", {"class":"prd"})

products = []
prices = []



#all_products_prices.write("products,prices\n")

for item in all_items:
    product_names = item.find("h3", {"class":"name"})
    product_prices = item.find("div", {"class":"prc"})
    
#     print(product_prices.text)   
    
    try:
        products.append(product_names.text.
                       replace(";", "").
                       replace(",", "")) 
        prices.append(int(product_prices.text.replace("KSh", "").replace(",", "")))
                
    except:
        continue
file = open("Products_and_Prices.csv", "w") 
file.write("product Names, Product Prices\n")
for product, price in zip(products, prices):
    product_prices = (price)
    product_names = (product)
    
file.write(f"{product_names}, {product_prices}\n")

        
        
file.close()
#print(products)
#print(prices)

In [3]:
all_products = list(zip(products, prices))

max_product = sorted(all_products, key = lambda products : products[1], reverse = True)[0]
min_product = sorted(all_products, key = lambda products : products[1], reverse = True)[-1]

print(max_product)
print(min_product)

('Samsung Galaxy M51 6.7" + 128GB + 8GB RAM (Dual SIM) 7000 MAh Black', 37599)
('Samsung Smart Phone Charger - White', 650)


In [4]:
product_df = pd.DataFrame(products)
product_df2 = pd.DataFrame(products)

#renaming column
product_df.columns = ['product_name']
product_df2.columns = ['product_name']

In [5]:
product_df['samsung type'] = product_df['product_name'].str[8:18]
product_df['screen size'] = product_df2['product_name'].str[18:24]
product_df['ram & rom'] = product_df2['product_name'].str[24:45]
product_df['description'] = product_df['product_name'].str[44:55]


In [73]:
new_product_df = product_df.iloc[: , 1:]
new_product_df.head()

,samsung type,screen size,ram & rom,description
0,Galaxy A12,- 6.5,″ - 128GB + 4GB RAM -,- Dual SIM-
1,Galaxy A12,– 6.5,″ – 64GB ROM + 4GB RA,AM – Dual S
2,Galaxy A12,"6.5""",4GB RAM + 128GB (Dual,l SIM) 5000
3,Galaxy A32,- 6.4,″ - 128GB ROM + 6GB R,RAM - Dual
4,Galaxy A12,"6.5""-",64GB/4GB-5000MAH(Dual,l SIM)-Blac


In [46]:
# converting series to list

samsung_list = new_product_df["samsung type"].tolist()
screen_list = new_product_df["screen size"].tolist()
ramrom_list = new_product_df["ram & rom"].tolist()
description_list = new_product_df["description"].tolist()

In [8]:
#convert to json

price = json.dumps(prices)

samsung_type = json.dumps(samsung_list)
screen_size = json.dumps(screen_list)
ram_rom = json.dumps(ramrom_list)
description = json.dumps(description_list)


In [74]:
#saving data to mysqldatabase

In [72]:
import mysql.connector
mydb = mysql.connector.connect(host="localhost",user="root",password="",database="jumia")
cursor = mydb.cursor()

for i in range(len(samsung_list)):
    price = prices[i] if i<len(prices) else 0 
    screen = screen_list[i] if i<len(screen_list) else "n/a"
    ramrom = ramrom_list[i] if i<len(ramrom_list) else "n/a"
    description = description_list[i] if i<len(description_list) else "n/a" 

    vals = (i,samsung_list[i]+str(i),price,screen,ramrom,description)
    
    sql = "INSERT INTO jumiaProducts(id,samsung_name,phone_price,screen_inches,ramrom,phone_description)VALUES(%s,%s,%s,%s,%s,%s)"

    cursor.execute(sql,vals)
    mydb.commit()
   